In [42]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [43]:
from scipy.spatial.distance import cosine
import operator

In [44]:
ratings = pd.read_csv(r"C:\Users\adity\Documents\Machine Learning\Recommendation\DataSets\rating.csv")
movies = pd.read_csv(r"C:\Users\adity\Documents\Machine Learning\Recommendation\DataSets\movie.csv")

In [45]:
df.shape

(20000263, 6)

In [46]:
df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
 4   title      object 
 5   genres     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB


In [48]:
df.duplicated().sum()

np.int64(0)

In [49]:
df = pd.merge(ratings, movies, on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [50]:
df.groupby('title')['rating'].mean().sort_values(ascending=False).head()

title
Tales That Witness Madness (1973)                        5.0
Stargate SG-1 Children of the Gods - Final Cut (2009)    5.0
Stealing a Nation (2004)                                 5.0
Abendland (2011)                                         5.0
Latin Music USA (2009)                                   5.0
Name: rating, dtype: float64

In [51]:
df.groupby('title')['userId'].count().sort_values(ascending=False).head()

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: userId, dtype: int64

In [52]:
n_movies = len((df['movieId'].unique()))
n_movies

26744

In [53]:
n_userid = len((df['userId'].unique()))
n_userid

138493

In [54]:
user_freq = df[['userId', 'movieId']].groupby('userId').count().reset_index()
user_freq.columns = ['userId', 'movieId']
user_freq

,userId,movieId
0,1,175
1,2,61
2,3,187
3,4,28
4,5,66
...,...,...
138488,138489,38
138489,138490,151
138490,138491,22
138491,138492,82


In [55]:
unique_genres = set()
for genres in df['genres']:
    genres_list = genres.split('|')
    for genre in genres_list:
        unique_genres.add(genre)


In [56]:
all_genres = sorted(list(unique_genres))
all_genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [57]:
def genres_to_vector(genres, all_genres):
    vector = [0] * len(all_genres)
    genres_list = genres.split('|')
    for genre in genres_list:
        if genre in all_genres:
            index = all_genres.index(genre)
            vector[index]= 1
    return vector

In [58]:
df['genres'][1]

'Adventure|Drama|Fantasy|Mystery|Sci-Fi'

In [59]:
genres_to_vector(df['genres'][1], all_genres)

[0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]

In [ ]:
movieDict = {}
for index, row in df.iterrows():
    movieId = row['movieId']
    title = row['title']
    genres_vector = genres_to_vector(row['genres'], all_genres)
    movieDict[movieId] = (title, genres_vector, ratings)

In [ ]:
movieDict[1]

In [ ]:
def computer_distance(movie1, movie2):
    genres_distance = cosine(movie1[1], movie2[1])
    rating_distance = abs(movie1[2] - movie2[2])
    total_distance = genres_distance + rating_distance
    return total_distance, genres_distance, rating_distance    

In [ ]:
def get_neighbours(movieId, k):
    distance = []
    for movie in movieDict:
        if movie != movieId:
            dist = computer_distance(movieDict[movieId], movieDict[movie])
            distance.append((movie, dist))
            
    distance.sort(key=operator.itemgetter(1))
    neighbours = []
    for i in range(min(k, len(distance))):
            neighbours.append(distance[i][0])
    return neighbours   


In [ ]:
movie_Id = int(input("Enter the movieId: "))
k = int(input("Enter the number of neighbours: "))
neighbours = get_neighbours(movie_Id, k)
average_rating = 0
for movie in neighbours:
    average_rating += movieDict[movie][2]
    print(movieDict[movie][0], movieDict[movie][2])
if neighbours:
    average_rating /= len(neighbours)
    print("Average rating of the neighbours: ", average_rating)
else:
    print("No neighbours found.")